In [3]:
import ipywebgl
import numpy as np
from ipywidgets import widgets, interact, interactive
from IPython.display import display, Javascript, HTML
import time

w = ipywebgl.GLViewer(width = 800, height = 800)
w.add_class('canvas')
w.clear_color(0.0, 0.0, 0.0, 1.0)
w.clear()
w.execute_commands()



# Vertex Shader & Fragment Shader

vertex_shader = w.create_shader('VERTEX_SHADER')
fragment_shader = w.create_shader('FRAGMENT_SHADER')

with open('vertex_shader.glsl', 'r') as file:
    vertex_source = file.read()


with open('fragment_shader.glsl', 'r') as file:
    fragment_source = file.read()

with open('BackgroundSetting.txt', 'r') as file:
    html_code = file.read()


w.shader_source(vertex_shader, vertex_source)
w.shader_source(fragment_shader, fragment_source)
w.compile_shader(vertex_shader)
w.compile_shader(fragment_shader)

program = w.create_program()
w.attach_shader(program, vertex_shader)
w.attach_shader(program, fragment_shader)
w.link_program(program)



# Buffer

Size = 3
Range = 1.0
x = np.linspace(-Range, Range, Size)
y = x
X, Y = np.meshgrid(x, y)
location = np.stack((X.flatten(), Y.flatten()),axis=1)

Index0 = []
for i in range(0, Size - 1):
    for j in range(0, Size - 1):
        Index0.append([i + j * Size, i + j * Size + 1, i + j * Size + Size])
        Index0.append([i + j * Size + 1, i + j * Size + Size, i + j * Size + Size + 1])
        
        
Index = np.asarray(Index0, dtype=np.uint8).flatten()

GridLocation = []
for i in Index:
    GridLocation.append(location[i])

vbo = w.create_buffer()

# bind the buffer and set the data in it
w.bind_buffer('ARRAY_BUFFER', vbo)
w.buffer_data(
    'ARRAY_BUFFER',
    np.array(GridLocation, dtype=np.float32),
    'STATIC_DRAW',
    update_info=True)

vao = w.create_vertex_array()
w.bind_vertex_array(vao)

w.use_program(program)
w.bind_buffer('ARRAY_BUFFER', vbo)
w.enable_vertex_attrib_array('in_position')
w.vertex_attrib_pointer('in_position', 2, "FLOAT", False, 2*4, 0)



# Core Generation using ipywidgets
w.clear()
w.use_program(program)
w.bind_vertex_array(vao)
start_time = time.time()
i = 0
def move(t, Dr0):
    global i
    global start_time
    w.uniform('t', np.asarray([t / 5.0], dtype=np.float32))
    w.uniform('amp', np.asarray([0.48], dtype=np.float32))
    w.uniform('Dr0', np.asarray([Dr0], dtype=np.float32))

    w.draw_arrays('TRIANGLES', 0, 3 * int(len(Index)/3))
    w.execute_commands()
    i += 1
    if i == 100:
        end_time = time.time()
        fps = 100 / (end_time - start_time)
        print(fps)
        start_time = end_time
        i = 0


play = widgets.Play(
    value=0,
    min=0,
    max=200,
    step=1,
    interval=16.67,
    description="play",
    disabled=False
)

# slider_t = widgets.FloatSlider(min=0, max=200, step=1, value=0, orientation='vertical')
# slider_Dr0 = widgets.FloatSlider(min=0, max=10, step=0.01, value=0, orientation='vertical')

slider_t = widgets.FloatSlider(min=0, max=200, step=1, value=0)
slider_Dr0 = widgets.FloatSlider(min=0, max=20, step=0.01, value=0)

widgets.jslink((play, 'value'), (slider_t, 'value'))

ui = widgets.HBox([play, slider_t, slider_Dr0])
out = widgets.interactive_output(move, {'t': slider_t, 'Dr0': slider_Dr0})

display(w, HTML(html_code), ui, out)

GLViewer(camera_pos=[0, 50, 200], height=800, width=800, _dom_classes=('canvas',))

Output()